## Adjust stock price such that call and put implied Volatility (IV) are the same
- calculate IV for nearest strike for both put and call
- if put IV > call IV then reduce stock price by the difference between put and call implied Volatility
- else if call IV > put IV then increase stock price by the difference between call and put implied Volatility
- keep adjusting stock price up or down by half the previous adjustment until call and put IV are about the same

Reason for IV differences between calls and puts is that the stock will normally drop by an amount slightly less than the dividend on xDividend date. Call IV should never by higher than put IV but we will allow for this unusual case (would be equivalent to a reverse dividend)


In [11]:
import datetime
import math
import mibian

In [12]:
def calc_call_iv(stock_price, strike_price, interest_rate, days_to_expiry, call_price):
    """
    calculate the implied volatility of a call option
    - return implied volatility relative to days_to_expiry (instead of annualized value)
    """
    bs = mibian.BS([stock_price, strike_price, interest_rate, days_to_expiry], callPrice=call_price)
    return bs.impliedVolatility / 100 * math.sqrt(days_to_expiry / 365) 


In [13]:
def calc_put_iv(stock_price, strike_price, interest_rate, days_to_expiry, put_price):
    """
    calculate the implied volatility of a put option
    - return implied volatility relative to days_to_expiry (instead of annualized value)
    """
    bs = mibian.BS([stock_price, strike_price, interest_rate, days_to_expiry], putPrice=put_price)
    return bs.impliedVolatility / 100 * math.sqrt(days_to_expiry / 365)

In [16]:
def center_stock_price(stock_price, strike_price, interest_rate, days_to_expiry, call_price, put_price):
    """
    calculate equilibrium implied volatility and the adjusted stock price at which it is acheived
    this is the stock price at which both call and put implied volatility are equal
    note: interest_rate needs be passed as a percent (e.g 5 = 5%)
    """
    #get starting call and put implied volatility
    call_iv = calc_call_iv(stock_price, strike_price, interest_rate, days_to_expiry, call_price)
    put_iv = calc_put_iv(stock_price, strike_price, interest_rate, days_to_expiry, put_price)
    iv_diff = abs(put_iv - call_iv)

    adj_lower = stock_price  * math.exp(-iv_diff) 
    adj_upper = stock_price  * math.exp(iv_diff)
    if put_iv > call_iv:
        #stock price is above equilibrium price
        adj_stock_price = (stock_price + adj_lower) / 2
    else:
        #stock price is below equilibrium price
        adj_stock_price = (stock_price + adj_upper) / 2

    adj_stock_price = stock_price 
    for i in range(100):
        call_iv = calc_call_iv(adj_stock_price, strike_price, interest_rate, days_to_expiry, call_price)
        put_iv = calc_put_iv(adj_stock_price, strike_price, interest_rate, days_to_expiry, put_price)
        iv_diff = abs(put_iv - call_iv)

        if iv_diff <= .0001:
            return adj_stock_price, call_iv, put_iv
        
        if put_iv > call_iv:
            #adjusted stock price is to high
            adj_upper = adj_stock_price
        else:
            #adjusted stock price is to low
            adj_lower = adj_stock_price
       
        adj_stock_price = (adj_lower + adj_upper) / 2

    #throw error if convergence was not achieved
    raise Exception("Put and Call implied volatilities did not converge")
        
        

In [17]:
#spy quote for July 28, 2020
#next xDividend date is September 18, 2020 (amount: 1.37)

stock_price = 321.20
quote_date = datetime.datetime(2020, 7, 28)
expiry_date =  datetime.datetime(2020, 10, 18)
strike_price = 321.0
call_bid = 13.31
call_ask = 13.38
put_bid = 14.05
put_ask = 14.12
interest_rate = 0 #use fed funds rate
call_price = (call_ask + call_bid) / 2
put_price =(put_bid + put_ask) / 2
days_to_expiry = (expiry_date -  quote_date).days 

adj_stock_price, call_iv, put_iv = center_stock_price(stock_price, strike_price, interest_rate, \
     days_to_expiry, call_price, put_price)

print("adj_stock_price:{0}, call_iv:{1}, put_iv:{2}".format(adj_stock_price, call_iv, put_iv))



adj_stock_price:320.2628225436524, call_iv:0.1072379482191231, put_iv:0.10725602914109562
